In [1]:
import os
import pandas as pd

from src.data.get import get_cryptocurrency_table

from private import coinmarketcap_apikey
from definitions import RAW_DATA_DIR, PROCESSED_DATA_DIR

cs_timeframe = '30m'

ModuleNotFoundError: No module named 'src.data.get'

In [6]:
crypto_table = get_cryptocurrency_table(
    apikey=coinmarketcap_apikey,
    start=1,
    limit=30,
    convert='USD'
)
crypto_table.to_csv(RAW_DATA_DIR / 'cryptocurrency_df.csv', index=False)

In [7]:
cryptocurrencies = []
stablecoins = []

for row in crypto_table.iterrows():
    if 'stablecoin' in row[1]['tags']:
        stablecoins.append(row[1]['symbol'])
    else:
        cryptocurrencies.append(row[1]['symbol'])
        
cryptocurrencies = [c.lower() for c in cryptocurrencies]
stablecoins = [s.lower() for s in stablecoins]

# We prefer USD quote since it has the longest time series in
# some cases. After that, prefer stablecoins in volume order.
preferred_quotes = ['usd'] + stablecoins

# Kraken is most preferred since it is the oldest exchange and
# it may be assumed that it has timeseries data reaching back
# in time the farthest. The subsequent exchanges are the ones
# ranked highest by CoinMarketCap. This selection is to ensure
# best data quality.
preferred_exchanges = [
    'kraken',
    'binance',
    'coinbase-pro',
    'bitfinex',
    'huobi',
    'uniswap-v2',
    'poloniex'
]

In [8]:
import cryptowatch as cw

all_markets = cw.markets.list().markets

all_markets_dic = {
    'id': [],
    'exchange': [],
    'pair': []
}

for market in all_markets:
    all_markets_dic['id'].append(market.id)
    all_markets_dic['exchange'].append(market.exchange)
    all_markets_dic['pair'].append(market.pair)

all_markets_df = pd.DataFrame(all_markets_dic)

In [14]:
results = {
    'id': [],
    'exchange': [],
    'base': [],
    'quote': []
}

for cc in cryptocurrencies:
    next_cc = False
    cc_markets = all_markets_df.loc[all_markets_df['pair'].str.startswith(cc)]
    for exchange in preferred_exchanges:
        if next_cc:
            break
        exch_markets = cc_markets[cc_markets['exchange'] == exchange]
        for quote in preferred_quotes:
            wanted_pair = exch_markets.loc[exch_markets['pair'].str[len(cc):] == quote]
            if len(wanted_pair) > 0:
                results['id'].append(wanted_pair.iloc[0]['id'])
                results['exchange'].append(exchange)
                results['base'].append(cc)
                results['quote'].append(quote)
                next_cc = True
                break
    if next_cc:
        continue
    # Base-quote pair not found in preferred exchanges -> look in all others
    wanted_pair = cc_markets.loc[cc_markets['pair'].str[len(cc):] == quote]
    for quote in preferred_quotes:
        wanted_pair = exch_markets.loc[exch_markets['pair'].str[len(cc):] == quote]
        if len(wanted_pair) > 0:
            results['id'].append(wanted_pair.iloc[0]['id'])
            results['exchange'].append(wanted_pair.iloc[0]['exchange'])
            results['base'].append(cc)
            results['quote'].append(quote)

chosen_pairs_df = pd.DataFrame(results)

In [15]:
chosen_pairs_df

,id,exchange,base,quote
0,87,kraken,btc,usd
1,96,kraken,eth,usd
2,2116,binance,bnb,usdt
3,5197,kraken,ada,usd
4,137,kraken,xrp,usd
5,61534,kraken,doge,usd
6,63657,kraken,dot,usd
7,92818,kraken,uni,usd
8,332334,binance,icp,usdt
9,146,kraken,bch,usd


In [16]:
# Check for duplicates
for ID in chosen_pairs_df['id']:
    if sum(chosen_pairs_df['id'] == ID) > 1:
        print(ID)

chosen_pairs_df[chosen_pairs_df['id'] == 61374]

,id,exchange,base,quote


In [20]:
candles = cw.markets.get(f'kraken:btcusd', ohlc=True, periods=['30m'])
candles.of_1d

AttributeError: 'MarketOHLCAPIResponse' object has no attribute 'of_1d'

In [21]:
dir(candles)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_allowance',
 '_fetched_at',
 '_http_response',
 '_legend',
 'of_30m']

In [22]:
getattr(candles, 'of_30m')

[[1620451800.0,
  58160.0,
  58160.0,
  57857.2,
  57956.2,
  49.91172573,
  2894212.042457093],
 [1620453600.0,
  57959.2,
  58028.5,
  57878.4,
  57941.6,
  39.24606032,
  2274279.304982753],
 [1620455400.0,
  57921.1,
  58029.0,
  57660.4,
  57825.6,
  27.63786074,
  1597765.631417978],
 [1620457200.0,
  57807.7,
  57846.1,
  57600.0,
  57825.0,
  24.7477042,
  1428929.446933047],
 [1620459000.0,
  57825.0,
  58148.7,
  57683.4,
  58128.9,
  35.90455084,
  2081104.760262598],
 [1620460800.0,
  58125.8,
  58600.0,
  58119.2,
  58535.7,
  78.48494418,
  4582605.821944251],
 [1620462600.0,
  58535.7,
  59420.0,
  58530.2,
  59240.0,
  268.15193393,
  15837429.265249014],
 [1620464400.0,
  59240.1,
  59395.0,
  58874.3,
  58933.1,
  65.26291991,
  3858615.543744354],
 [1620466200.0,
  58933.1,
  59199.9,
  58713.5,
  59180.2,
  86.13517654,
  5077304.486937566],
 [1620468000.0,
  59180.3,
  59180.3,
  58759.8,
  58889.1,
  44.64432162,
  2630294.329935856],
 [1620469800.0,
  58889.1,
  

In [26]:
os.mkdir(RAW_DATA_DIR / cs_timeframe)
for i, row in enumerate(chosen_pairs_df.iterrows()):
    _id, exchange, base, quote = (
        row[1]['id'], row[1]['exchange'], row[1]['base'], row[1]['quote']
    )
    try:
        candles = cw.markets.get(f'{exchange}:{base}{quote}', ohlc=True, periods=[cs_timeframe])
        candles_df = pd.DataFrame(
            getattr(candles, f'of_{cs_timeframe}'),
            columns=[
                'timestamp', 'open', 'high', 'low', 'close',
                'volume', 'volume_quote'
            ]
        )
        candles_df.to_csv(RAW_DATA_DIR / cs_timeframe / f'{exchange}_{base}_{quote}.csv', index=False)
        print('Saved successfully:')
        print(f'  {i+1:03}/{len(chosen_pairs_df)} {exchange}:{base}{quote}')
    except Exception as e:
        print(e)

Saved successfully:
  001/25 kraken:btcusd
Saved successfully:
  002/25 kraken:ethusd
Saved successfully:
  003/25 binance:bnbusdt
Saved successfully:
  004/25 kraken:adausd
Saved successfully:
  005/25 kraken:xrpusd
Saved successfully:
  006/25 kraken:dogeusd
Saved successfully:
  007/25 kraken:dotusd
Saved successfully:
  008/25 kraken:uniusd
Saved successfully:
  009/25 binance:icpusdt
Saved successfully:
  010/25 kraken:bchusd
Saved successfully:
  011/25 kraken:linkusd
Saved successfully:
  012/25 kraken:ltcusd
Saved successfully:
  013/25 kraken:maticusd
Saved successfully:
  014/25 kraken:xlmusd
Saved successfully:
  015/25 kraken:etcusd
Saved successfully:
  016/25 binance:solusdt
Saved successfully:
  017/25 binance:vetusdt
Saved successfully:
  018/25 binance:thetausdt
Saved successfully:
  019/25 coinbase-pro:wbtcusd
Saved successfully:
  020/25 kraken:eosusd
Saved successfully:
  021/25 kraken:trxusd
Saved successfully:
  022/25 kraken:filusd
Saved successfully:
  023/25 kr

In [4]:
from datetime import datetime as dt

In [5]:
ts1 = 1546300800000
ts2 = 1548979140000
print(f'{ts1}: {dt.fromtimestamp(ts1)}')
print(f'{ts2}: {dt.fromtimestamp(ts2)}')

OSError: [Errno 22] Invalid argument

In [10]:
dt.fromtimestamp(1548979140)

datetime.datetime(2019, 2, 1, 0, 59)